In [2]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd
import kss
import re
import faiss
import torch

In [7]:
#pip install faiss-gpu

     |████████████████████████████████| 85.5 MB 125.1 MB/s eta 0:00:01  |█▍                              | 3.8 MB 6.9 MB/s eta 0:00:12
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install faiss-cpu

     |████████████████████████████████| 27.0 MB 7.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [17]:
model_path = '/home/u4026/KoSentenceBERT_SKTBERT/output/training_sts'
model = SentenceTransformer(model_path)
#model = SentenceTransformer("distiluse-base-multilingual-cased-v1")

using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_v1.zip
using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
Load Model


/home/u4026/.local/lib/python3.8/site-packages/sentence_transformers/models/Transformer.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.auto_model.load_state_dict(

In [4]:
nli_model_path = '/home/u4026/KoSentenceBERT_SKTBERT/output/training_nli'
nli_model = SentenceTransformer(nli_model_path)

using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_v1.zip
using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
Load Model


/home/u4026/.local/lib/python3.8/site-packages/sentence_transformers/models/Transformer.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.auto_model.load_state_dict(

In [4]:
#pip show transformers

Name: transformers
Version: 4.46.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /home/u4026/.local/lib/python3.8/site-packages
Requires: regex, filelock, pyyaml, tokenizers, huggingface-hub, safetensors, requests, numpy, tqdm, packaging
Required-by: sentence-transformers, kobert
Note: you may need to restart the kernel to use updated packages.


In [5]:
# CSV 파일 로드
csv_file = "books_summary.csv"
data = pd.read_csv(csv_file)

In [25]:
def split_and_filter_sentences(text):               #무시하자
    """
    텍스트를 KSS와 추가 기준으로 분리하고, 의미 없는 연결어를 제거
    """
    # KSS로 1차 분리
    sentences = kss.split_sentences(text)
    
    # 추가 기준 적용 ('그리고', '또', ',' 기준으로 분리)
    split_sentences = []
    for sentence in sentences:
        parts = re.split(r'(그리고|또|,)', sentence)
        parts = [p.strip() for p in parts if p.strip()]  # 공백 제거 및 빈 문자열 제거
        split_sentences.extend(parts)
    
    # 불필요한 연결어 필터링
    filtered_sentences = [s for s in split_sentences if s not in ["그리고", "또", ","]]
    
    return filtered_sentences

In [21]:
def get_average_embedding(sentences, model):        #무시하자
    """
    여러 문장의 임베딩 값을 평균화
    """
    if not sentences:
        return None  # 빈 입력에 대한 처리

    embeddings = model.encode(sentences, convert_to_tensor=True)
    avg_embedding = embeddings.mean(dim=0)  # 평균 계산
    return avg_embedding

In [6]:
# 전처리된 텍스트 열 추출
corpus = data['내용요약'].tolist()
titles = data['첵제목'].tolist()

In [26]:
# 코퍼스 문장 분리 및 임베딩 평균 계산                무시하자
corpus_embeddings = []
for doc in corpus:
    sentences = split_and_filter_sentences(doc)
    avg_embedding = get_average_embedding(sentences, model)
    corpus_embeddings.append(avg_embedding)

In [7]:
# NLI 코퍼스 임베딩 생성
corpus_embeddings = nli_model.encode(corpus, convert_to_tensor=True)

In [26]:
# STS 코퍼스 임베딩 생성
sts_corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

In [8]:
embedding_dimension = corpus_embeddings.shape[1]

In [27]:
sts_embedding_dimension = sts_corpus_embeddings.shape[1]

In [ ]:
print("상위 5개 '내용요약' 데이터:")
print(corpus[:5]) 

In [28]:
# FAISS CPU 설정
index = faiss.IndexFlatL2(embedding_dimension)  # L2 거리 기반 인덱스 생성
index.add(sts_corpus_embeddings.cpu().numpy())

In [10]:
# NLI를 활용해 포함 관계 분석
def find_entailments_with_batches(query, corpus, titles, batch_size=500, top_k=3, threshold=0.2):
    results = []
    query_embedding = nli_model.encode([query], convert_to_tensor=True)

    for i in range(0, len(corpus), batch_size):
        batch_corpus = corpus[i:i + batch_size]
        batch_titles = titles[i:i + batch_size]
        batch_embeddings = nli_model.encode(batch_corpus, convert_to_tensor=True)

        # 코사인 유사도 계산 (NLI 모델 활용)
        similarities = util.pytorch_cos_sim(query_embedding, batch_embeddings)

        for j, similarity in enumerate(similarities[0]):
            score = similarity.item()
            if score >= threshold:  # Threshold 적용
                results.append({
                    "책제목": batch_titles[j],
                    "내용요약": batch_corpus[j],
                    "포함확률": score
                })

    # 상위 N개 결과 정렬
    return sorted(results, key=lambda x: x["포함확률"], reverse=True)[:top_k]


In [29]:
#faiss 써서 포함관계분석
def find_entailments_with_faiss(query, corpus, titles, top_k=3, threshold=1.0):
    results = []
    query_embedding = model.encode([query], convert_to_tensor=True).cpu().numpy()  # GPU 텐서를 CPU NumPy 배열로 변환

    # FAISS에서 최근접 이웃 검색
    distances, indices = index.search(query_embedding, top_k)

    for dist, idx in zip(distances[0], indices[0]):
        score = 1 - dist  # 거리 값을 유사도로 변환
        if score <=threshold:  # Threshold 적용
            results.append({
                "책제목": titles[idx],
                "내용요약": corpus[idx],
                 "포함확률": 1 - dist if dist < 1 else 0,  # 거리 값을 유사도로 변환
                "L2 거리": dist
                #"포함확률": score
            })

    return sorted(results, key=lambda x: x["포함확률"], reverse=True)[:top_k]

In [19]:
# NLI모델로 평가
def evaluate_with_nli(query, entailment_results):
    query_embedding = nli_model.encode([query], convert_to_tensor=True)
    results = []

    for result in entailment_results:
        corpus_embedding = nli_model.encode([result['내용요약']], convert_to_tensor=True)
        similarity = util.pytorch_cos_sim(query_embedding, corpus_embedding).item()
        results.append({
            "책제목": result['책제목'],
            "내용요약": result['내용요약'],
            "포함확률": result['포함확률'],
            "유사도": similarity
        })

    # 유사도를 기준으로 정렬
    return sorted(results, key=lambda x: x["유사도"], reverse=True)

In [18]:
# STS 모델로 평가
def evaluate_with_sts(query, entailment_results):
    query_embedding = model.encode([query], convert_to_tensor=True)
    results = []

    for result in entailment_results:
        corpus_embedding = model.encode([result['내용요약']], convert_to_tensor=True)
        similarity = util.pytorch_cos_sim(query_embedding, corpus_embedding).item()
        results.append({
            "책제목": result['책제목'],
            "내용요약": result['내용요약'],
            "포함확률": result['포함확률'],
            "유사도": similarity
        })

    # 유사도를 기준으로 정렬
    return sorted(results, key=lambda x: x["유사도"], reverse=True)

In [14]:
# Step 2: 쿼리 입력 및 상위 3개 검색           이거안씀
def search_nearest_neighbors(query, top_k=3):
    query_embedding = model.encode([query], convert_to_tensor=True).cpu().numpy()  # GPU 텐서를 NumPy 배열로 변환
    
    # FAISS에서 상위 3개 검색
    distances, indices = index.search(query_embedding, top_k)
    
    # 검색된 3개의 데이터 반환
    nearest_data = [{"index": idx, "내용요약": corpus[idx], "distance": dist}
                    for dist, idx in zip(distances[0], indices[0])]
    return nearest_data


In [16]:
# Step 3: 검색된 데이터와 쿼리 상세 비교       이거 안씀
def compare_with_nearest(query, nearest_data):
    query_embedding = model.encode([query], convert_to_tensor=True).cpu()  # GPU 텐서를 CPU 텐서로 변환
    
    # 검색된 데이터 임베딩 추출
    selected_embeddings = np.array([corpus_embeddings[entry["index"]].cpu().numpy() for entry in nearest_data])
    
    # 유사도 계산 (코사인 유사도)
    similarities = util.pytorch_cos_sim(query_embedding, selected_embeddings)
    
    # 유사도 추가
    for i, entry in enumerate(nearest_data):
        entry["유사도"] = similarities[0][i].item()
    return sorted(nearest_data, key=lambda x: x["유사도"], reverse=True)

In [23]:
# 사용자가 입력한 텍스트
query = input("텍스트를 입력하세요: ")


# 쿼리 임베딩 생성
#query_embedding = nli_model.encode([query], convert_to_tensor=True)

텍스트를 입력하세요: 새 교육과정과 관련된 책 추천해줘


In [30]:
# NLI를 사용해 상위 3개 필터링
#entailment_results = find_entailments_with_batches(query, corpus, titles, batch_size=500, top_k=3)

# faiss를 사용해 상위 3개 필터링
entailment_results = find_entailments_with_faiss(query,corpus,titles, top_k=3)

# NLI로 유사도 평가
#final_results = evaluate_with_nli(query, entailment_results)

#STS로 유사도 평가
final_results = evaluate_with_sts(query, entailment_results)

# 결과 출력
print("\n가장 관련성 높은 책 추천:")
for result in final_results:
    print(f"책제목: {result['책제목']}")
    print(f"내용요약: {result['내용요약']}")
   # print(f"포함확률: {result['포함확률']:.4f}")
    print(f"유사도: {result['유사도']:.4f}\n")



가장 관련성 높은 책 추천:
책제목: 교육광장 2016 겨울호 (Vol. 62)
내용요약: 새 교육과정에 따른 평가기준의 개발이 필요해서, 한국교육과정평가원에서는 총론과 교과별 평가기준 등을 연구 개발 하였다.  핵심 개발 방향은 2015년 개정 교육과정이 학교현장에서 운영되도록 하는 것이다.
유사도: 0.5778

책제목: 행복한 교육
내용요약: 한국교육학술정보원은 개별 대학에서 공개하는 강의를 수집하여 서비스하기 시작했으며, 서울대, 카이스트, 연세대, 성균관대, 울산과학기술대, 숙명여대 등이 참여하기 시작했고, 국가는 이를 평생교육 차원에서 K-MOOC 서비스로 구축할 계획을 세우고 있다.
유사도: 0.4919

책제목: 국제 학습성과 지표 개발을 위한 학습성과 명세화 및 국제 비교 방안 탐색
내용요약: 학생들의 읽기 문식성에 대한 연구인 PIRLS는 4학년을 대상으로 한다. 2011년에 4, 5, 6학년을 대상으로 Pre-PIRLS을 구성했다. 따라서 2006년 PIRLS와 2011 Pre-PIRLS의 평가 요소별 비율은 다소 다르다.
유사도: 0.4631



In [22]:
# 함수 호출           이거 안씀
nearest_data = search_nearest_neighbors(query, top_k=3)
final_results = compare_with_nearest(query, nearest_data)

# 결과 출력
print("\n가장 유사한 3개의 문장:")
for idx, result in enumerate(final_results):
    print(f"Rank {idx + 1}:")
    print(f"내용요약: {result['내용요약']}")
    print(f"유사도: {result['유사도']:.4f}\n")


가장 유사한 3개의 문장:
Rank 1:
내용요약: 연구반은 지역 MBC, KBS 지역총국의 편성‧경영 현황을 검토하고 관계자의 자체제작물 편성의무 의견과 논의를 통해 제도적 실현가능성과 세부사항을 논의했다. 
유사도: 0.5576

Rank 2:
내용요약: PD는 자격유지를 위해 기획재정부 고시 규정상 의무를 이행해야하는 바, 이행 실적에 따라 점수로 평가를 받게되고, PPD는 특성상 보다 적은 의무를 부과한다.
유사도: 0.5176

Rank 3:
내용요약: 한국산업기술평가관리원은 국가 혁신역량제고를 목적으로 설립되었다. 기관장은 설립목적에 부합하는 다양한 경영방침을 수립하고 신핵심가치 설정과 이를 실행하기 위한 내부 실행력을 강화하고 공공기관의 사회적 책임을 다하기 위해 노력하는 등의 모습을 보였다.
유사도: 0.4974



In [28]:
# 코사인 유사도 비교         #이거 안씀
if query_embedding is not None:
    cos_scores = [util.pytorch_cos_sim(query_embedding, corpus_embedding)[0].item() for corpus_embedding in corpus_embeddings]

    # 상위 유사도 결과 출력
    top_k = min(5, len(cos_scores))
    top_results = np.argsort(cos_scores)[-top_k:][::-1]  # 유사도 높은 순으로 정렬

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop {} most similar sentences in corpus:".format(top_k))

    for idx in top_results:
        print(f"{corpus[0]} (Score: {cos_scores[idx]:.4f})")
else:
    print("입력 텍스트에서 유효한 문장이 없습니다.")





Query: 요즘 우리나라 대통령이 개엄령을 일으켜서 놀라웠는데 예전에 있었던 광주시위에 관한책을 소개시켜줘 광주시위후 슬픔을 받는 사람들의 이야기였으면 해

Top 3 most similar sentences in corpus:
소설은 6개의 장으로 나누어져 있었는데요 각 장마다 다른 인물의 시선으로 이야기가 진행되고 있었어요 주인공들은 광주에서 일어난 시위 사건 속에 있던 사람들로 각자의 슬픔을 안고 살아가고 있습니다 (Score: 0.4742)
소설은 6개의 장으로 나누어져 있었는데요 각 장마다 다른 인물의 시선으로 이야기가 진행되고 있었어요 주인공들은 광주에서 일어난 시위 사건 속에 있던 사람들로 각자의 슬픔을 안고 살아가고 있습니다 (Score: 0.2166)
소설은 6개의 장으로 나누어져 있었는데요 각 장마다 다른 인물의 시선으로 이야기가 진행되고 있었어요 주인공들은 광주에서 일어난 시위 사건 속에 있던 사람들로 각자의 슬픔을 안고 살아가고 있습니다 (Score: 0.1448)


In [2]:
# 코사인 유사도 계산
cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
cos_scores = cos_scores.cpu()

# 책 제목별 최고 유사도 선택
best_results = {}
for idx, score in enumerate(cos_scores):
    title = titles[idx]
    sentence = corpus[idx].strip()
    if title not in best_results or best_results[title]['score'] < score.item():
        best_results[title] = {'sentence': sentence, 'score': score.item()}

    # 유사도 점수가 높은 순으로 정렬
sorted_results = sorted(best_results.items(), key=lambda x: x[1]['score'], reverse=True)

# 가장 유사한 결과 가져오기
#top_k = 3
#top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

# 결과 출력
print("\n\n======================\n\n")
print("Query:", query)
print("\nTop 5 most similar sentences in corpus:")
for title, result in sorted_results[:top_k]:
        print(f"Title: {title}, Sentence: {result['sentence']} (Score: {result['score']:.4f})")

#for idx in top_results[0:top_k]:
    #print(f"Title: {titles[idx]}, Sentence: {corpus[idx].strip()} (Score: {cos_scores[idx]:.4f})")
    #print(f"{corpus[idx].strip()} (Score: {cos_scores[idx]:.4f})")

NameError: name 'util' is not defined

In [6]:
corpus = ['갑자기 테이블 바깥쪽에 앉아있던 피해자 C(여, 39세)의 왼쪽 가슴을 3회 만져 피해자를 강제로 추행하였다.',
          '주거지에 찾아와 "같이 살자"라고 말을 하며 피해자의 상의를 걷어 올려 양손으로 피해자의 양쪽 가슴을 수차례 주무른 후 피해자의 손을 잡아채 피고인의 성기에 가져가 피고인의 성기를 만지게 하고, 이어 피해자의 바지 위에 손을 얹어 피해자의 음부를 만졌다.\\n2. 2020. 8. 8.자 범행\\n피고인은 2020. 8. 8. 09:00경 제1항의 피해자 주거지에서, 마당에 있다가 집 안으로 들어가는 피해자를 따라 들어가 피해자의 상의 옷 안으로 한 손을 넣어 가슴을 주무르고, 피해자의 바지 속에 손을 넣어 피해자의 음부를 만졌다.\\n이로써 피고인은 피해자를 강제로 추행하였다.',
          "피해자 D(가명)에게 다가가 ‘자기야'라고 말하면서 피해자 왼편에 서서 오른 팔로 피해자의 어깨를 감싸 안아 피해자를 강제로 추행하였다.",
          '피고인은 종업원인 피해자 C(여, 21세)와 이야기를 하면서 갑자기 손으로 피해자의 가슴을 3회 가량 쳐 피해자를 강제로 추행하였다.',
          "피해자에게 ‘한번 하자, 안 해주면 안 간다. 아는 사람을 시켜서 장사 못하게 한다.'라는 등의 말을 하면서 성관계를 요구하였으나 피해자가 이를 거절하자 갑자기 피해자의 엉덩이와 가슴을 손으로 수회 움켜잡고, 이를 뿌리치고 일어나는 피해자에게 ‘한번 안아주면 가겠다.'라고 말하여 이에 피해자가 어쩔 수 없이 응하자 그 틈을 타 피해자의 팬티 안으로 손을 집어넣어 피해자의 음부를 수회 움켜쥐는 등 피해자를 강제로 추행하였다."
          '피고인과 피해자 B(여, 37세)는 학교 선후배 사이로 알고 지내는 관계이다. 식당의 1, 2층을 연결하는 내부 계단 중간 부분에서, 피해자와 마주치자 갑자기 손으로 피해자의 허리를 끌어당겨 안으면서 피해자에게 "이러면 내가 따먹는다"라고 말하여 피해자를 강제로 추행하였다.',
          '차량 안에서 갑자기 피해자의 머리를 세게 잡아당겨 입술에 키스하였고, 같은 날 23:50경 같은 시에 있는 번지불상의 도로를 진행하는 위 차량 안에서 운전 중인 피해자의 가슴을 만지려고 시도하여 피해자가 피고인의 손을 쳐내면서 "뭐하는 거냐, 술집 여자도 아닌데?"라고 항의하는데도 "가슴이 예쁘다, 하나도 안 꺼졌다"라고 말하면서 피해자의 블라우스 안으로 손을 집어넣고 가슴을 주물렀다.거부하는데도 "나랑 만나는 거 생각해 봐, 남편한테 너를 빼앗아 오겠다, 익산에 가서 먹겠다"라고 말하면서 피해자의 블라우스 안으로 손을 집어 넣고 가슴을 여러 번 주물렀다.하차하려는 피해자를 제지하면서 머리를 잡아당겨 입술에 키스하고, 피해자의 블라우스를 세게 잡아당겨 단추가 뜯어지자 피해자의 가슴을 입으로 빨고, "모텔에 같이 가자"고 말하면서 피해자의 바지 후크를 풀어 지퍼를 내리고 손을 집어넣어 피해자의 음부를 만졌다.\\n이로써 피고인은 피해자를 추행하였다.',
          '피고인의 집에서, 피고인의 부탁으로 기타를 가르쳐 주러 온 피해자가 피고인의 방침대에 걸터앉아 휴대폰 충전을 하고 있는 것을 보고 피해자의 뒤로 다가가, 갑자기 양팔로 피해자의 허리를 껴안으면서 "나 섹스하고 싶어"라고 말하고, 이에 피해자가 피고인의 손을 뿌리치자, 양팔을 빼는 과정에서 갑자기 손으로 피해자의 허벅지 부위를 쓸어내리듯이 만지고, 이에 피해자가 피고인을 피하여 자리에서 일어나자, 피해자의 손목을 잡아 피해자를 침대에 다시 앉힌 다음 피해자의 뒤에서 갑자기 양팔로 피해자의 허리를 껴안으면서 피해자에게 "진짜 한번만 해 주면 안 돼?"라고 말하고, 계속하여 피고인을 피하여 자리에서 일어나 나가려고 하는 피해자의 팔을 끌어 당겨 피해자가 침대 앞에 있는 피아노 의자에 앉자, 피해자에게 "너 손 작다."라고 말하면서 갑자기 피해자의 손에 깍지를 끼고, 이에 또다시 피해자가 손을 뺐다. 계속하여 피고인은 피해자에게"나 진짜 못 참겠으니까 한번만 하자"라고 하면서 갑자기 피해자의 손목을 잡아당겼다.\\n이로써 피고인은 피해자를 강제로 추행하였다.',
          "술을 마시던 중 자리에서 일어나 피해자에게 다가가 오른손을 피해자 옷 속에 집어넣어 왼쪽 가슴을 4회 만졌다.이로써 피고인은 피해자를 강제로 추행하였다.",
          '피고인의 옆에서 수영 강습을 받기 위해 물에 엎드린 자세로 떠 있던 피해자의 엉덩이를 갑자기 손으로 움켜쥐었다.이로써 피고인은 피해자를 강제로 추행하였다.']

corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

query = ['피고인은 엉덩이를 손으로 움켜쥐어 피해자를 강제로 추행하였다.']

query_embedding = model.encode(query, convert_to_tensor=True)

cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
cos_scores = cos_scores.cpu()

top_k = 5

top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

print("\n\n======================\n\n")
print("Query:", query)
print("\nTop 5 most similar sentences in corpus:")

for idx in top_results[0:top_k]:
  print(corpus[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))





Query: ['피고인은 엉덩이를 손으로 움켜쥐어 피해자를 강제로 추행하였다.']

Top 5 most similar sentences in corpus:
피고인의 옆에서 수영 강습을 받기 위해 물에 엎드린 자세로 떠 있던 피해자의 엉덩이를 갑자기 손으로 움켜쥐었다.이로써 피고인은 피해자를 강제로 추행하였다. (Score: 0.7840)
피해자 D(가명)에게 다가가 ‘자기야'라고 말하면서 피해자 왼편에 서서 오른 팔로 피해자의 어깨를 감싸 안아 피해자를 강제로 추행하였다. (Score: 0.7435)
술을 마시던 중 자리에서 일어나 피해자에게 다가가 오른손을 피해자 옷 속에 집어넣어 왼쪽 가슴을 4회 만졌다.이로써 피고인은 피해자를 강제로 추행하였다. (Score: 0.7283)
갑자기 테이블 바깥쪽에 앉아있던 피해자 C(여, 39세)의 왼쪽 가슴을 3회 만져 피해자를 강제로 추행하였다. (Score: 0.7032)
피고인은 종업원인 피해자 C(여, 21세)와 이야기를 하면서 갑자기 손으로 피해자의 가슴을 3회 가량 쳐 피해자를 강제로 추행하였다. (Score: 0.7030)
